In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.0.1


In [2]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

# Tutorial 6  
Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[paper](https://arxiv.org/pdf/1611.07308.pdf)  
[code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

## Graph AutoEncoder GAE

### Load the data

In [3]:
dataset = Planetoid("/Users/wciezobka/sano/projects/masters-thesis/Datasets", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!
/Users/wciezobka/venvs/sano/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [4]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [5]:
data = train_test_split_edges(data)

/Users/wciezobka/venvs/sano/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

### Define the Encoder

In [7]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define the Autoencoder

In [8]:
from torch_geometric.nn import GAE

In [9]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [10]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)



In [11]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6099, AP: 0.6496
Epoch: 002, AUC: 0.6462, AP: 0.6750
Epoch: 003, AUC: 0.6507, AP: 0.6809
Epoch: 004, AUC: 0.6537, AP: 0.6847
Epoch: 005, AUC: 0.6561, AP: 0.6877
Epoch: 006, AUC: 0.6580, AP: 0.6907
Epoch: 007, AUC: 0.6592, AP: 0.6926
Epoch: 008, AUC: 0.6594, AP: 0.6938
Epoch: 009, AUC: 0.6602, AP: 0.6962
Epoch: 010, AUC: 0.6609, AP: 0.6996
Epoch: 011, AUC: 0.6610, AP: 0.7020
Epoch: 012, AUC: 0.6603, AP: 0.7044
Epoch: 013, AUC: 0.6599, AP: 0.7071
Epoch: 014, AUC: 0.6589, AP: 0.7091
Epoch: 015, AUC: 0.6576, AP: 0.7112
Epoch: 016, AUC: 0.6570, AP: 0.7135
Epoch: 017, AUC: 0.6556, AP: 0.7150
Epoch: 018, AUC: 0.6549, AP: 0.7166
Epoch: 019, AUC: 0.6539, AP: 0.7173
Epoch: 020, AUC: 0.6533, AP: 0.7178
Epoch: 021, AUC: 0.6526, AP: 0.7185
Epoch: 022, AUC: 0.6521, AP: 0.7189
Epoch: 023, AUC: 0.6517, AP: 0.7192
Epoch: 024, AUC: 0.6513, AP: 0.7195
Epoch: 025, AUC: 0.6512, AP: 0.7199
Epoch: 026, AUC: 0.6512, AP: 0.7203
Epoch: 027, AUC: 0.6512, AP: 0.7208
Epoch: 028, AUC: 0.6520, AP:

In [13]:
Z = model.encode(x, train_pos_edge_index)
Z.shape

torch.Size([3327, 2])

## Are the results (AUC) and (AP) easy to read and compare?

# Use Tensorboard

In [15]:
from torch.utils.tensorboard import SummaryWriter

In [28]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Import tensorboard

#### Installation: (if needed) "pip install tensorboard"

In [29]:
writer = SummaryWriter('../../Logs/GAE1_experiment_'+'2d_100_epochs')

In [30]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6490, AP: 0.6802
Epoch: 002, AUC: 0.6524, AP: 0.6859
Epoch: 003, AUC: 0.6573, AP: 0.6915
Epoch: 004, AUC: 0.6601, AP: 0.6946
Epoch: 005, AUC: 0.6612, AP: 0.6960
Epoch: 006, AUC: 0.6622, AP: 0.6974
Epoch: 007, AUC: 0.6636, AP: 0.6989
Epoch: 008, AUC: 0.6642, AP: 0.7000
Epoch: 009, AUC: 0.6652, AP: 0.7011
Epoch: 010, AUC: 0.6648, AP: 0.7018
Epoch: 011, AUC: 0.6647, AP: 0.7027
Epoch: 012, AUC: 0.6647, AP: 0.7042
Epoch: 013, AUC: 0.6648, AP: 0.7061
Epoch: 014, AUC: 0.6649, AP: 0.7085
Epoch: 015, AUC: 0.6646, AP: 0.7112
Epoch: 016, AUC: 0.6643, AP: 0.7137
Epoch: 017, AUC: 0.6636, AP: 0.7161
Epoch: 018, AUC: 0.6633, AP: 0.7187
Epoch: 019, AUC: 0.6627, AP: 0.7200
Epoch: 020, AUC: 0.6623, AP: 0.7215
Epoch: 021, AUC: 0.6617, AP: 0.7228
Epoch: 022, AUC: 0.6614, AP: 0.7237
Epoch: 023, AUC: 0.6610, AP: 0.7244
Epoch: 024, AUC: 0.6610, AP: 0.7253
Epoch: 025, AUC: 0.6608, AP: 0.7255
Epoch: 026, AUC: 0.6607, AP: 0.7258
Epoch: 027, AUC: 0.6608, AP: 0.7262
Epoch: 028, AUC: 0.6607, AP:

## Graph Variational AutoEncoder (GVAE)

In [21]:
from torch_geometric.nn import VGAE

In [22]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

/home/antonio/anaconda3/envs/geometric_new/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [23]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [24]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [25]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6043, AP: 0.6264
Epoch: 002, AUC: 0.4937, AP: 0.5243
Epoch: 003, AUC: 0.4943, AP: 0.5001
Epoch: 004, AUC: 0.5013, AP: 0.5079
Epoch: 005, AUC: 0.5011, AP: 0.5006
Epoch: 006, AUC: 0.5022, AP: 0.5011
Epoch: 007, AUC: 0.5022, AP: 0.5011
Epoch: 008, AUC: 0.5022, AP: 0.5011
Epoch: 009, AUC: 0.4978, AP: 0.5007
Epoch: 010, AUC: 0.4978, AP: 0.5007
Epoch: 011, AUC: 0.4978, AP: 0.5007
Epoch: 012, AUC: 0.5011, AP: 0.5011
Epoch: 013, AUC: 0.5011, AP: 0.5011
Epoch: 014, AUC: 0.5011, AP: 0.5011
Epoch: 015, AUC: 0.5011, AP: 0.5011
Epoch: 016, AUC: 0.5011, AP: 0.5011
Epoch: 017, AUC: 0.5011, AP: 0.5011
Epoch: 018, AUC: 0.5011, AP: 0.5011
Epoch: 019, AUC: 0.5011, AP: 0.5011
Epoch: 020, AUC: 0.5011, AP: 0.5011
Epoch: 021, AUC: 0.5011, AP: 0.5011
Epoch: 022, AUC: 0.4989, AP: 0.4995
Epoch: 023, AUC: 0.4989, AP: 0.4995
Epoch: 024, AUC: 0.4989, AP: 0.4995
Epoch: 025, AUC: 0.4989, AP: 0.4995
Epoch: 026, AUC: 0.4989, AP: 0.4995
Epoch: 027, AUC: 0.4989, AP: 0.4995
Epoch: 028, AUC: 0.4989, AP:

Epoch: 231, AUC: 0.4989, AP: 0.4995
Epoch: 232, AUC: 0.4989, AP: 0.4995
Epoch: 233, AUC: 0.4989, AP: 0.4995
Epoch: 234, AUC: 0.4989, AP: 0.4995
Epoch: 235, AUC: 0.4989, AP: 0.4995
Epoch: 236, AUC: 0.4989, AP: 0.4995
Epoch: 237, AUC: 0.4989, AP: 0.4995
Epoch: 238, AUC: 0.4989, AP: 0.4995
Epoch: 239, AUC: 0.4989, AP: 0.4995
Epoch: 240, AUC: 0.4989, AP: 0.4995
Epoch: 241, AUC: 0.5011, AP: 0.5011
Epoch: 242, AUC: 0.5011, AP: 0.5011
Epoch: 243, AUC: 0.5011, AP: 0.5011
Epoch: 244, AUC: 0.5011, AP: 0.5011
Epoch: 245, AUC: 0.5011, AP: 0.5011
Epoch: 246, AUC: 0.5011, AP: 0.5011
Epoch: 247, AUC: 0.5011, AP: 0.5011
Epoch: 248, AUC: 0.5011, AP: 0.5011
Epoch: 249, AUC: 0.5011, AP: 0.5011
Epoch: 250, AUC: 0.5011, AP: 0.5011
Epoch: 251, AUC: 0.5011, AP: 0.5011
Epoch: 252, AUC: 0.5011, AP: 0.5011
Epoch: 253, AUC: 0.5011, AP: 0.5011
Epoch: 254, AUC: 0.5011, AP: 0.5011
Epoch: 255, AUC: 0.5011, AP: 0.5011
Epoch: 256, AUC: 0.5011, AP: 0.5011
Epoch: 257, AUC: 0.5011, AP: 0.5011
Epoch: 258, AUC: 0.5011, AP: